## First: Review Existing Unstructured Data and Diagram a New Structured Relational Data Model

In [1]:
import pandas as pd
import numpy as np
import ast
import json

In [2]:
# Get data from json files
brands_orig = pd.read_json('brands.json')
receipts_orig = pd.read_json('receipts.json')
users_orig = pd.read_json('users.json')

In [3]:
# date_convert function to convert date columns from timestamp to date
def date_convert(date):
    if pd.isna(date) == True:
        return(np.nan)
    else:
        return(pd.to_datetime((int(date['$date'])/1000), unit='s'))

# Data cleaning for receipts. Modify the type of date data
date_list = ['createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate']

for date in date_list:
    receipts_orig[date] = receipts_orig[date].apply(lambda x:date_convert(x)).values.astype('datetime64[s]')
    
receipts_orig['_id'] = receipts_orig['_id'].apply(lambda x: x['$oid'])

# rewardsReceiptItemList contains multiple dictionaries. The column needs to be splited to make each row only has one item
receipts_rewards = receipts_orig.explode('rewardsReceiptItemList').reset_index(drop=True).fillna({'rewardsReceiptItemList':'{}'})
receipts_rewards['rewardsReceiptItemList'] = receipts_rewards['rewardsReceiptItemList'].astype(str).apply(lambda x: ast.literal_eval(x))
receipts_rewards_split = pd.json_normalize(receipts_rewards['rewardsReceiptItemList']).add_prefix('rewardsReceiptItemList.')
receipts = receipts_rewards.merge(receipts_rewards_split, left_index=True, right_index=True)
receipts

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,...,rewardsReceiptItemList.itemNumber,rewardsReceiptItemList.originalMetaBriteQuantityPurchased,rewardsReceiptItemList.pointsEarned,rewardsReceiptItemList.targetPrice,rewardsReceiptItemList.competitiveProduct,rewardsReceiptItemList.originalFinalPrice,rewardsReceiptItemList.originalMetaBriteItemPrice,rewardsReceiptItemList.deleted,rewardsReceiptItemList.priceAfterCoupon,rewardsReceiptItemList.metabriteCampaignId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,5.0,2021-01-03 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,603d0b710a720fde1000042a,NaN,NaN,2021-03-01 15:42:41,2021-03-01 15:42:41,NaT,2021-03-01 15:42:41,NaT,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7377,603cf5290a720fde10000413,NaN,NaN,2021-03-01 14:07:37,2021-03-01 14:07:37,NaT,2021-03-01 14:07:37,NaT,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7378,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28,2021-03-01 13:07:28,NaT,2021-03-01 13:07:29,NaT,25.0,2020-08-17 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
7379,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28,2021-03-01 13:07:28,NaT,2021-03-01 13:07:29,NaT,25.0,2020-08-17 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN


In [4]:
# Data cleaning for brands
brands_orig['_id'] = brands_orig['_id'].apply(lambda x: x['$oid'])
brands_cpg = pd.json_normalize(brands_orig['cpg']).add_prefix('cpg.')
brands = brands_orig.merge(brands_cpg, left_index=True, right_index=True).drop(columns=['cpg'])
brands

,_id,barcode,category,categoryCode,name,topBrand,brandCode,cpg.$ref,cpg.$id.$oid
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7
...,...,...,...,...,...,...,...,...,...
1162,5f77274dbe37ce6b592e90c0,511111116752,Baking,BAKING,test brand @1601644365844,NaN,NaN,Cogs,5f77274dbe37ce6b592e90bf
1163,5dc1fca91dda2c0ad7da64ae,511111706328,Breakfast & Cereal,NaN,Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL,Cogs,53e10d6368abd3c7065097cc
1164,5f494c6e04db711dd8fe87e7,511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217,Cogs,5332fa12e4b03c9a25efd1e7
1165,5a021611e4b00efe02b02a57,511111400608,Grocery,NaN,LIPTON TEA Leaves,0.0,LIPTON TEA Leaves,Cogs,5332f5f6e4b03c9a25efd0b4


In [5]:
# Data cleaning for users
users_orig['_id'] = users_orig['_id'].apply(lambda x: x['$oid'])
users_orig['createdDate'] = users_orig['createdDate'].apply(lambda x:date_convert(x)).values.astype('datetime64[s]')
users_orig['lastLogin'] = users_orig['lastLogin'].apply(lambda x:date_convert(x)).values.astype('datetime64[s]')
users = users_orig
users

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
491,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
492,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
493,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN


**Note: For the structured and relational diagram, please review the Relational_diagram.png.**

## Second: Write queries that directly answer predetermined questions from a business stakeholder

### 1.What are the top 5 brands by receipts scanned for most recent month?

In [6]:
createDate = receipts.sort_values(by='createDate', ascending=False)
latest_date = createDate['createDate'].reset_index(drop=True).loc[0]
receipts_last_month = receipts[receipts['createDate'] >= latest_date - pd.Timedelta(days=29)]
receipts_last_month.groupby('rewardsReceiptItemList.brandCode')['rewardsReceiptItemList.brandCode'].count().reset_index(name='count')\
.sort_values(['count'], ascending=False)

# SQL code:
# select rewardsReceiptItemList.brandCode, count(*) cnt
# from receipts
# where createDate >= date_sub('2021-03-01', interval 29 day)
# group by rewardsReceiptItemList.brandCode
# order by cnt desc
# limit 5

,rewardsReceiptItemList.brandCode,count
0,BRAND,3
1,MISSION,2
2,VIVA,1


Answer: There are only three brands for most recent month. The brandCodes are "BRAND", "MISSION", "VIVA".

### 2.How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

In [7]:
receipts_previous_month = receipts[(receipts['createDate'] <= latest_date - pd.Timedelta(days=29)) & \
                                   (receipts['createDate'] >= latest_date - pd.Timedelta(days=60))]
receipts_previous_month.groupby('rewardsReceiptItemList.brandCode')['rewardsReceiptItemList.brandCode'].count().reset_index(name='count')\
.sort_values(['count'], ascending=False).head(5).reset_index(drop=True)

# SQL code:
# select rewardsReceiptItemList.brandCode, count(*) cnt
# from receipts
# where createDate >= date_sub('2021-03-01', interval 60 day)
# group by rewardsReceiptItemList.brandCode
# order by cnt desc
# limit 5

,rewardsReceiptItemList.brandCode,count
0,HY-VEE,291
1,BEN AND JERRYS,180
2,PEPSI,93
3,KROGER,89
4,KLEENEX,88


Answer: Top 5 brands for the previous month are "HY-VEE", "BEN AND JERRYS", "PEPSI", "KROGER", "KLEENEX".

### 3.When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [8]:
receipts_orig.groupby('rewardsReceiptStatus')['totalSpent'].mean().reset_index(name='avg_spent')

# SQL code:
# select rewardsreceiptstatus, avg(totalspent) avg_spent from receipts group by rewardsreceiptstatus

,rewardsReceiptStatus,avg_spent
0,FINISHED,80.854305
1,FLAGGED,180.451739
2,PENDING,28.032449
3,REJECTED,23.326056
4,SUBMITTED,NaN


Answer: The average spent for "Accepted" ("Finished" in table) is greater.

### 4.When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [9]:
receipts_orig.groupby('rewardsReceiptStatus')['purchasedItemCount'].sum().reset_index(name='total_items')

# SQL code:
# select rewardsreceiptstatus, sum(purchaseditemcount) tot_purchase_cnt from receipts group by rewardsreceiptstatus

,rewardsReceiptStatus,total_items
0,FINISHED,8184.0
1,FLAGGED,1014.0
2,PENDING,0.0
3,REJECTED,173.0
4,SUBMITTED,0.0


Answer: The total number of items purchased for "Accepted" ("Finished" in table) is greater.

### 5.Which brand has the most spend among users who were created within the past 6 months?

In [10]:
user_createDate = users.sort_values(by='createdDate', ascending=False)
user_latest_date = user_createDate['createdDate'].reset_index(drop=True).loc[0]
users_six_months = users[users['createdDate'] >= user_latest_date - pd.DateOffset(months=6)].drop_duplicates(subset=['_id'], keep='first')

receipts['rewardsReceiptItemList.finalPrice'] = receipts['rewardsReceiptItemList.finalPrice'].astype(float)
receipts_six_months = receipts[receipts['userId'].isin(users_six_months['_id'])]
receipts_six_months.groupby('rewardsReceiptItemList.brandCode')['rewardsReceiptItemList.finalPrice'].sum().reset_index(name='total_price')\
.sort_values(by='total_price', ascending=False).reset_index(drop=True)

# SQL code:
# select rewardsReceiptItemList.brandCode, sum(rewardsReceiptItemList.finalPrice) total_price
# from receipts
# where userId in (
#     select distinct _id from users where createddate >= date_sub('2021-02-12', interval 6 month)
# )
# group by rewardsReceiptItemList.brandCode
# order by total_price desc

,rewardsReceiptItemList.brandCode,total_price
0,BEN AND JERRYS,1217.40
1,HEMPLER'S,800.55
2,CRACKER BARREL,703.50
3,HY-VEE,656.62
4,KNORR,543.32
...,...,...
162,ROSARITA,1.25
163,MORTON,1.19
164,DANNON,1.12
165,KARO,0.60


Answer: "BEN AND JERRYS" has the most spend among users who were created within the past 6 months.

### 6.Which brand has the most transactions among users who were created within the past 6 months?

In [11]:
receipts_six_months.groupby('rewardsReceiptItemList.brandCode')['rewardsReceiptItemList.finalPrice'].count().reset_index(name='trans_count')\
.sort_values(by='trans_count', ascending=False).reset_index(drop=True)

# SQL code:
# select rewardsReceiptItemList.brandCode, count(_id) trans_count
# from receipts
# where userId in (
#     select distinct _id from users where createddate >= date_sub('2021-02-12', interval 6 month)
# )
# group by rewardsReceiptItemList.brandCode
# order by trans_count desc

,rewardsReceiptItemList.brandCode,trans_count
0,HY-VEE,291
1,BEN AND JERRYS,100
2,PEPSI,74
3,KLEENEX,70
4,KNORR,60
...,...,...
162,SCHWEBEL'S,1
163,HERITAGE FARM,1
164,SIMPLE TRUTH,1
165,GREY POUPON,1


Answer: "HY-VEE" has the most transactions among users who were created within the past 6 months.

## Third: Evaluate Data Quality Issues in the Data Provided

1.There are too much missing data in three files, even a lot of important and necessary data is missing. For example, 234 brands do not have a BrandCode. It will be difficult to join the brands and receipts tables if some BrandCodes are missing.

In [12]:
brands[brands['brandCode'].isna()==True].shape[0]

234

2.Receipts data for a long period of time is missing. It can be clearly seen from the result of the following code that the date from 2021-02-13 to 2021-02-25 is not in the receipts table. This directly results in that only three brands could be counted in February.

In [13]:
pd.to_datetime(receipts_last_month['createDate']).dt.date.drop_duplicates(keep='first')

6776    2021-01-31
6797    2021-02-01
6830    2021-02-02
6894    2021-02-03
6928    2021-02-04
6959    2021-02-05
6989    2021-02-06
7019    2021-02-07
7049    2021-02-08
7079    2021-02-09
7110    2021-02-10
7157    2021-02-11
7189    2021-02-12
7220    2021-02-26
7253    2021-02-27
7294    2021-02-28
7338    2021-03-01
Name: createDate, dtype: object

3.Duplicate rows are found in the users dataset.There are total 495 rows in the users file. After removing all duplicate rows, only 212 rows remain, which means that more than half of the rows are duplicates.

In [14]:
users.drop_duplicates(subset=['_id'], keep='first').shape[0]

212

4.Some brands from the receipts cannot be found in the Brands file. There are total 186 brands in the receipts that are missing in the Brands table. When we do queries between these two tables, there would be so many missing records.

In [15]:
receipts[~receipts['rewardsReceiptItemList.brandCode'].isin(brands['brandCode'])]['rewardsReceiptItemList.brandCode'].drop_duplicates(keep='first').shape[0]

186

5.rewardsReceiptItemList column in the Receipts file has a JSON list format, which is too hard to analyze. When I answered those questions from business stakeholders, the first step is to split the JSON data. But this is a time-consuming work and cannot be done every time. So the data should be stored with each item instead of a list.

## Fourth: Communicate with Stakeholders

Hi everyone,

I hope you all are doing well. I am glad to share my findings and questions after analyzing three datasets, including Brands, Receipts, and Users. All three files have JSON formats. I used Python to transfer the JSON data to DataFrame, then cleaned the data and got some business results. During this process, I detected some data quality issues which may impact the accuracy of my analysis. Below are my questions and data issues I have found.

- Process to discover the data quality issues

When I analyzed the top 5 brands by receipts scanned in Feb 2021, I found that the number of BrandCode is less than 10. I thought that one possible reason was the missing data of BrandCode. Another reason is that some dates in Feb are lost. I filted distinct dates in Feb, then found that dates from 2021-02-13 to 2021-02-25 are not in the table. The result proved my assumption. For the Users file, it was easy to find those duplicate IDs when I cleaned the data. I also discovered that all the dates are Dictionary data type. I wrote a function to transfer them to the date format to simplify my analysis.<br />

- Questions I have about the data and more information I need

Some data issues are easy to solve, such as duplicate rows in the Users file. But for other issues like much missing data of BrandCode, this is a serious problem when I need to join tables. I expected to find an ID field in both Brands and Receipts tables to be the foreign key, but the only column I could use is the BrandCode. What worse is that over one hundred brand names could not be found in the Brands table. I assumed that the Brands file should contain all the brands. So the most critical question is if the two JSON files are complete, and how should I join two datasets? What I need is additional data, such as BrandCodes and missing dates in Feb. I can handle other issues when I do the data cleaning.<br />

- Performance and scaling concerns

The most important concern is about the data type of rewardsReceiptItemList, which is a list containing one or more dictionaies. I am not sure if this is an unchangeable storage method. If not, then I would suggest each record only has one item instead of the whole receipt. The reason is that splitting the receipt is one more step before the analysis. It might lead to additional issues or mistakes. When the dataset is large enough, updating the data type is also a time-consuming work. The same issue for the ID related data. These may be potential problems in the future.

Please review the analysis report I attached. I would be very grateful if you can provide help with my questions and concerns. And if you have any questions, please let me know. Thank you.

Best regards,<br />
Puyi
